# Group 10 ETL Project

Import all packs

In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
import pandas as pd

Grab the URL and scrape the data needed

In [2]:
engine = create_engine("sqlite:///Resources/mlsseasons.sqlite", echo=False)

In [3]:
engine.execute("drop table seasons")
for year in range(2007, 2019):
    url = f"https://www.mlssoccer.com/standings/supporters-shield/{year}/"
    print(url)
    tables = pd.read_html(url)
    df = tables[0]
    df.columns = df.loc[df.index[0]]
    df.drop([0],inplace= True)
    df.set_index('#', inplace=True)
    df["season"] = year
    df.to_sql("seasons", con=engine, if_exists='append')

engine.execute("select count(*) from seasons").first()

https://www.mlssoccer.com/standings/supporters-shield/2007/
https://www.mlssoccer.com/standings/supporters-shield/2008/
https://www.mlssoccer.com/standings/supporters-shield/2009/
https://www.mlssoccer.com/standings/supporters-shield/2010/
https://www.mlssoccer.com/standings/supporters-shield/2011/
https://www.mlssoccer.com/standings/supporters-shield/2012/
https://www.mlssoccer.com/standings/supporters-shield/2013/
https://www.mlssoccer.com/standings/supporters-shield/2014/
https://www.mlssoccer.com/standings/supporters-shield/2015/
https://www.mlssoccer.com/standings/supporters-shield/2016/
https://www.mlssoccer.com/standings/supporters-shield/2017/
https://www.mlssoccer.com/standings/supporters-shield/2018/


(218,)

Read salaries CSV file to pandas and write to SQL database

In [4]:
sal = pd.read_csv("Resources/MLS_Salaries.csv", encoding='UTF-16', sep="\t")


In [5]:
sal.to_sql("salaries", con=engine, if_exists='replace')
engine.execute("select count(*) from salaries").first()

(6219,)

create a mapping table for joining salaries data to seasons data 

In [7]:
engine.execute("drop table club_map")
engine.execute("create table club_map (long_name varchar(255), short_name varchar(255))")

In [8]:

engine.execute("""
insert into club_map values
('DCD.C. United','DC'),
('CHVChivas USA','CHV'),
('HOUHouston Dynamo','HOU'),
('NENew England Revolution','NE'),
('DALFC Dallas','FCD'),
('RBNYNew York Red Bulls','RBNY'),
('SKCSporting Kansas City','KC'),
('CHIChicago Fire FC','CHI'),
('CLBColumbus Crew SC','CLB'),
('COLColorado Rapids','COL'),
('LALA Galaxy','LAG'),
('RSLReal Salt Lake','RSL'),
('TORToronto FC','TFC'),
('SJSan Jose Earthquakes','SJ'),
('SEASeattle Sounders FC','SEA'),
('PHIPhiladelphia Union','PHI'),
('PORPortland Timbers','POR'),
('VANVancouver Whitecaps FC','VAN'),
('MTLMontreal Impact','MTL'),
('ORLOrlando City SC','ORL'),
('NYCNew York City FC','NYC'),
('ATLAtlanta United FC','ATL'),
('MINMinnesota United FC','MNU'),
('LAFCLos Angeles Football Club','LAFC')
""")

In [10]:
sample=pd.DataFrame(engine.execute("""
select s."First Name", s."Last Name",
c.Club, c.pts, c.season
from seasons c
join club_map m on m.long_name = c.Club
join salaries s on s."Club (grouped)" = m.short_name
where c.season = s.Season
"""),columns=['First Name','Last Name','Club','PTS','Season'])
sample


,First Name,Last Name,Club,PTS,Season
0,None,Carreiro da Silva Fred,DCD.C. United,55,2007
1,Ben,Olsen,DCD.C. United,55,2007
2,Brad,North,DCD.C. United,55,2007
3,Brian,Carroll,DCD.C. United,55,2007
4,Bryan,Arguez,DCD.C. United,55,2007
...,...,...,...,...,...
6206,Quincy,Amarikwa,SJSan Jose Earthquakes,21,2018
6207,Shea,Salinas,SJSan Jose Earthquakes,21,2018
6208,Tommy,Thompson,SJSan Jose Earthquakes,21,2018
6209,"Valeri ""Vako""",Qazaishvili,SJSan Jose Earthquakes,21,2018
